# Core SSEBroadcastManager

> Core SSE broadcast management system for FastHTML applications. Provides connection pooling, message distribution, and lifecycle hooks without UI dependencies.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import asyncio
from typing import Set, Dict, Any, Optional, Callable, Deque
from collections import deque
from datetime import datetime
import json

In [ ]:
#| export
class SSEBroadcastManager:
    """Manages SSE connections and broadcasting without UI dependencies."""
    
    def __init__(self, 
                 max_queue_size: int = 100,    # Maximum number of messages per connection queue
                 history_size: int = 50,    # Number of broadcast messages to keep in history
                 default_timeout: float = 0.1   # Default timeout in seconds for queue operations
                ):
        """Initialize the broadcast manager with connection pooling and message history."""
        self.connections: Set[asyncio.Queue] = set()
        self.lock = asyncio.Lock()
        self.history: Deque[Dict[str, Any]] = deque(maxlen=history_size)
        self.max_queue_size = max_queue_size
        self.default_timeout = default_timeout
        
        # Event hooks for extensibility
        self._on_connect_hooks: list[Callable] = []
        self._on_disconnect_hooks: list[Callable] = []
        self._on_broadcast_hooks: list[Callable] = []
    
    async def register_connection(
        self,
        queue: Optional[asyncio.Queue] = None  # Optional pre-existing queue, creates new one if not provided
    ) -> asyncio.Queue:  # The queue associated with this connection
        """Register a new SSE connection."""
        if queue is None:
            queue = asyncio.Queue(maxsize=self.max_queue_size)
        
        async with self.lock:
            self.connections.add(queue)
            
        # Call connection hooks
        for hook in self._on_connect_hooks:
            try:
                await hook(queue) if asyncio.iscoroutinefunction(hook) else hook(queue)
            except Exception as e:
                print(f"Error in connection hook: {e}")
        
        return queue
    
    async def unregister_connection(
        self,
        queue: asyncio.Queue  # The queue to unregister
    ):
        """Unregister an SSE connection."""
        async with self.lock:
            self.connections.discard(queue)
        
        # Call disconnection hooks
        for hook in self._on_disconnect_hooks:
            try:
                await hook(queue) if asyncio.iscoroutinefunction(hook) else hook(queue)
            except Exception as e:
                print(f"Error in disconnection hook: {e}")
    
    async def broadcast(self, 
                       event_type: str,   # Type of event being broadcast
                       data: Dict[str, Any], # Data to broadcast
                       timeout: Optional[float] = None # Optional timeout override for this broadcast
                       ) -> int: # Number of successfully notified connections
        """Broadcast a message to all connected clients."""
        message = {
            "type": event_type,
            "timestamp": datetime.now().isoformat(),
            "data": data
        }
        
        # Store in history
        self.history.append(message)
        
        # Call broadcast hooks
        for hook in self._on_broadcast_hooks:
            try:
                result = await hook(message) if asyncio.iscoroutinefunction(hook) else hook(message)
                # Allow hooks to modify the message
                if isinstance(result, dict):
                    message = result
            except Exception as e:
                print(f"Error in broadcast hook: {e}")
        
        # Broadcast to all connections
        timeout = timeout or self.default_timeout
        successful_sends = 0
        
        async with self.lock:
            disconnected = set()
            
            for queue in self.connections:
                try:
                    await asyncio.wait_for(queue.put(message), timeout=timeout)
                    successful_sends += 1
                except (asyncio.TimeoutError, asyncio.QueueFull):
                    disconnected.add(queue)
                except Exception as e:
                    print(f"Error broadcasting to connection: {e}")
                    disconnected.add(queue)
            
            # Remove disconnected clients
            for queue in disconnected:
                self.connections.discard(queue)
        
        return successful_sends
    
    def on_connect(
        self,
        callback: Callable  # Function to call when a new connection is registered
    ):
        """Register a callback for new connections."""
        self._on_connect_hooks.append(callback)
        return callback
    
    def on_disconnect(
        self,
        callback: Callable  # Function to call when a connection is unregistered
    ):
        """Register a callback for disconnections."""
        self._on_disconnect_hooks.append(callback)
        return callback
    
    def on_broadcast(
        self,
        callback: Callable  # Function to call before broadcasting (can modify messages)
    ):
        """Register a callback for broadcasts (can modify messages)."""
        self._on_broadcast_hooks.append(callback)
        return callback
    
    @property
    def connection_count(
        self
    ) -> int:  # Number of active connections
        """Get the current number of active connections."""
        return len(self.connections)
    
    def get_history(
        self,
        limit: Optional[int] = None  # Optional limit on number of messages to return
    ) -> list[Dict[str, Any]]:  # List of historical broadcast messages
        """Get broadcast history."""
        if limit:
            return list(self.history)[-limit:]
        return list(self.history)

In [ ]:
# Initialize the SSE Broadcast Manager
sse_manager = SSEBroadcastManager(
    max_queue_size=100,
    history_size=50,
    default_timeout=0.1
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()